## doctest from psyclone.transformations.OMPLoopTrans

In [1]:
from psyclone.parse.algorithm import parse
from psyclone.parse.utils import ParseError
from psyclone.psyGen import PSyFactory, GenerationError

In [2]:
api="gocean1.0"
filename = "alg.f90"
ast, invokeInfo = parse(filename, api=api, invoke_name="invoke")
psy = PSyFactory(api).create(invokeInfo)
print(psy.invokes.names)

dict_keys(['invoke_0'])


In [12]:
from psyclone.psyGen import TransInfo
t = TransInfo()
ltrans = t.get_trans_name('OMPLoopTrans')
rtrans = t.get_trans_name('OMPParallelTrans')
trans = t.get_trans_name('OMPParallelLoopTrans')

In [4]:
schedule = psy.invokes.get('invoke_0').schedule
schedule.view()
new_schedule = schedule

GOInvokeSchedule[invoke='invoke_0', Constant loop bounds=True]
    Loop[type='outer', field_space='go_cu', it_space='go_internal_pts']
        Literal[value:'2']
        Literal[value:'jstop']
        Literal[value:'1']
        Schedule[]
            Loop[type='inner', field_space='go_cu', it_space='go_internal_pts']
                Literal[value:'2']
                Literal[value:'istop+1']
                Literal[value:'1']
                Schedule[]
                    CodedKern compute_cu_code(cu_fld,p_fld,u_fld) [module_inline=False]
    Loop[type='outer', field_space='go_cv', it_space='go_internal_pts']
        Literal[value:'2']
        Literal[value:'jstop+1']
        Literal[value:'1']
        Schedule[]
            Loop[type='inner', field_space='go_cv', it_space='go_internal_pts']
                Literal[value:'2']
                Literal[value:'istop']
                Literal[value:'1']
                Schedule[]
                    CodedKern compute_cv_code(cv_fld,p_fld,v_

Apply the OpenMP Loop transformation to *every* loop in the schedule

In [5]:
for child in schedule.children:
    newschedule, memento = ltrans.apply(child, reprod=True)
    schedule = newschedule

Enclose all of these loops within a single OpenMP PARALLEL region

In [15]:
newschedule, memento = rtrans.apply(schedule.children)
print(newschedule.dag_name)
print(memento)

schedule


# OMPParallelLoopTrans

In [24]:
from psyclone.transformations import OMPParallelLoopTrans

In [25]:
ast, invokeInfo = parse("dynamo.F90")
psy = PSyFactory("dynamo0.1").create(invokeInfo)
schedule = psy.invokes.get('invoke_v3_kernel_type').schedule
schedule.view()

ParseError: "Parse Error: In the dynamo0.3 API the 1st argument of a meta_arg entry should be a valid argument type (one of ['gh_field', 'gh_operator', 'gh_columnwise_operator', 'gh_real', 'gh_integer']), but found 'gh_rw' in 'arg_type(gh_rw, v3, fe, .true., .false., .true.)'"

In [20]:
trans = OMPParallelLoopTrans()
new_schedule, memento = trans.apply(schedule.children[0])

TransformationError: 'Transformation Error: Error in OMPParallelLoopTrans transformation. The node is not a loop.'